# 1. Forewords
## In this notebook, we will use and showcase the capabilities of Gemma 3n running locally via Ollama.

**Let's try to use the model as much as possible without using GPU. This shows that any computer with few resources can run our code.**

# 2. Package Installation Script

This section of the code is responsible for safely installing the required Python packages for the project. It defines a function to perform package installations while checking for existing installations to avoid redundancy.

## Key Features:

- **Dynamic Package Management**: The script can install specific versions of packages or the latest available version from the Python Package Index (PyPI) and Git repositories.

- **Error Handling**: The installation function captures any exceptions that occur during the installation process, providing feedback in case of any failures.

- **Clear Dependencies**: A dictionary (`required_packages`) lists all required packages along with their desired versions, making it easy to manage dependencies.

## Usage:
1. The function `install_package` checks whether a package is already installed and installs it if not.
2. Each package in the `required_packages` dictionary is processed, and the installation procedure is invoked, ensuring all necessary libraries are available for the application to function properly.

This approach helps maintain a clean and organized environment essential for smooth development and deployment of the project.

In [1]:
# Function to safely install packages
def install_package(package, version=None):
    """Install a package safely, checking if it's already installed."""
    try:
        if version:
            # Use >= for minimum version specification
            !pip install -q {package}{version} 
        else:
            !pip install -q {package} 
        print(f"Successfully installed {package}")
        return True
    except Exception as e:
        print(f"Failed to install {package}: {e}")
        return False

# List of required packages with versions
required_packages = {
    "gradio": "latest",
    "ollama": "latest",
    "torch": ">=2.4.0",
    "transformers": ">=4.53.0"
    
}

# Install packages
for package, version in required_packages.items():
    if version == "latest":
        install_package(package)
    else:
        install_package(package, version)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.6 MB/s eta 0:00:00


# 3. Imports

In [2]:
%%time
import os
import sys
import psutil
import subprocess
import logging
import warnings
import gradio as gr
import ollama
from ollama import chat
from PIL import Image
import io
import base64
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText
!pip list | grep -E 'torch|transformers'

2025-06-30 18:41:30.714351: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751308890.957424      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751308891.025512      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch-ignite                     0.5.2
pytorch-lightning                  2.5.1.post0
sentence-transformers              3.4.1
torch                              2.6.0+cu124
torchao                            0.10.0
torchaudio                         2.6.0+cu124
torchdata                          0.11.0
torchinfo                          1.8.0
torchmetrics                       1.7.1
torchsummary                       1.5.1
torchtune                          0.6.1
torchvision                        0.21.0+cu124
transformers                       4.51.3
CPU times: user 24.1 s, sys: 4 s, total: 28.1 s
Wall time: 41.4 s


# 4. Logger Configuration

This section of code is responsible for setting up a logging system in Python, enabling the application to record log messages at various levels (INFO, ERROR, etc.). Logging is essential for debugging and monitoring applications.

## Key Components:

1. **Logger Creation**:
   - A logger is created using `logging.getLogger(__name__)`, which allows tracking logs specific to the current module or script.

2. **Log Level Setting**:
   - The logger’s level is set to `INFO` using `logger.setLevel(logging.INFO)`, which means that all messages at this level and above (WARNING, ERROR, CRITICAL) will be captured.

3. **File Handler**:
   - A file handler is created with `logging.FileHandler('app.log')`, which writes log messages to a specified file (in this case, `app.log`).
   - It is configured to capture messages at the INFO level and above.

4. **Console Handler**:
   - A console handler is set up with `logging.StreamHandler()`, which outputs log messages to the console (standard output).
   - This handler is configured to capture messages only at the ERROR level and above, which helps to reduce console clutter during regular operations.

5. **Formatters**:
   - The format of the log messages is defined using `logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')`. This format includes:
     - Timestamp of the log message (`asctime`)
     - Name of the logger (`name`)
     - Level of the log message (`levelname`)
     - The log message itself (`message`)

6. **Adding Handlers**:
   - The configured file and console handlers are added to the logger with `logger.addHandler()`, allowing the logger to output messages to both the console and a log file with the specified formats.

In [3]:
# Cell: Logger Configuration
# Create a logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler and set level to debug
file_handler = logging.FileHandler('app.log')
file_handler.setLevel(logging.INFO)

# Create console handler and set level to error
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.ERROR)

# Create formatters
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Add formatters to handlers
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add handlers to logger
logger.addHandler(file_handler)
logger.addHandler(console_handler)

# 5. System Information Function

This section of code defines a function `print_system_info()` that prints detailed information about the current system environment, including Python version, current working directory, PyTorch version, and GPU availability.


In [4]:
# ==================================================
# _/$\_\%/_/&\_\@/_/$\_\%/_/&\_\@/_/$\_\%/_/&\_\@/_
# ==================================================
#  **********    System Information   *************
# ==================================================
# _/$\_\%/_/&\_\@/_/$\_\%/_/&\_\@/_/$\_\%/_/&\_\@/_
# ==================================================

def print_system_info():
    print("System Information:")
    print(f"• Python version: {sys.version}")
    print(f"• Current working directory: {os.getcwd()}")
    print(f"• PyTorch version: {torch.__version__}")

    # Check GPU availability and details
    if torch.cuda.is_available():
        gpu_info = {
            "CUDA Available": torch.cuda.is_available(),
            "CUDA Device Count": torch.cuda.device_count(),
            "Current CUDA Device": torch.cuda.current_device(),
            "Device Name": torch.cuda.get_device_name(torch.cuda.current_device()),
            "Memory Allocated (MB)": round(torch.cuda.memory_allocated(0) / 1024**2, 2),
            "Memory Reserved (MB)": round(torch.cuda.memory_reserved(0) / 1024**2, 2),
        }
        
        print("\n⚡ GPU Detected:")
        for key, value in gpu_info.items():
            print(f"  • {key}: {value}")
    else:
        print("\n😭 No GPU detected. Running on CPU only.")

    # Memory information
    ram = psutil.virtual_memory()
    print("\n🐘 System Memory:")
    print(f"  • Total RAM: {round(ram.total / 1024**2, 2)} MB")
    print(f"  • Available RAM: {round(ram.available / 1024**2, 2)} MB")
    print(f"  • Used RAM: {round(ram.used / 1024**2, 2)} MB")
    print(f"  • RAM Percentage: {ram.percent}% used")

print_system_info()

System Information:
• Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
• Current working directory: /kaggle/working
• PyTorch version: 2.6.0+cu124

😭 No GPU detected. Running on CPU only.

🐘 System Memory:
  • Total RAM: 32102.9 MB
  • Available RAM: 30376.23 MB
  • Used RAM: 1269.46 MB
  • RAM Percentage: 5.4% used


# 6. Ollama Installation Script

This command is used to install the Ollama software on your system by downloading and executing the installation script.

## Command Breakdown:

- **`!curl -fsSL https://ollama.com/install.sh`**:
  - `curl`: A command-line tool for transferring data with URLs. It is used here to download the installation script from the specified URL.
  - `-f`: This option tells `curl` to fail silently on server errors (like 404 or 500 HTTP responses), which means it won't proceed to execute if there are issues accessing the script.
  - `-s`: This flag makes the `curl` operation silent, meaning no progress meter or error messages will be shown.
  - `-S`: This option tells `curl` to show errors if they occur, even if the `-s` (silent) option is used.
  - `-L`: This flag tells `curl` to follow redirects. If the URL is redirected to another location, `curl` will follow it to download the file.

- **`| sh`**:
  - The pipe `|` takes the output of the `curl` command (the contents of the `install.sh` script) and passes it to the shell (`sh`) for execution.
  - This allows the script to be run directly after being downloaded without the need to save it to a file first.


In [5]:
%%time
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
CPU times: user 1 s, sys: 288 ms, total: 1.29 s
Wall time: 45.9 s


# 7. Running Ollama Server with subprocess

This command is used to start the Ollama server as a subprocess within a Python script. The server allows you to interact with the Ollama model API.

## Command Breakdown:

- **`subprocess.Popen`**:
  - `subprocess`: This is a Python module used to spawn new processes, connect to their input/output/error pipes, and obtain their return codes.
  - `Popen`: A constructor from the `subprocess` module that executes a child program in a new process. It can be used to execute shell commands and interact with them.

- **Parameters**:
  - **`"ollama serve"`**: This is the command being executed. In this case, it starts the Ollama server, which provides an interface for interacting with the Ollama model.
  - **`shell=True`**: This option indicates that the command should be executed through the shell. It allows for shell-specific features, such as running commands like `ollama serve` directly as if you were typing it into the command line.


In [6]:
%%time
process = subprocess.Popen("ollama serve", shell=True)

CPU times: user 947 µs, sys: 14 µs, total: 961 µs
Wall time: 725 µs


# 8. Pulling the Gemma 3n Model

This command is used to download the `gemma3n:e2b` model from the Ollama repository to your local environment. It ensures that you have the necessary model files for running inference or training tasks.

## Command Breakdown:

- **`!ollama`**:
  - This is the command-line interface (CLI) for interacting with the Ollama service. Using the `!` prefix indicates that this command is being run in a shell interface, often seen in Jupyter notebooks and some interactive environments.

- **`pull`**:
  - The `pull` command is used to download a specified model from the Ollama repository. This retrieves the model's weights and configuration files, making them available for local use.

- **`gemma3n:e2b`**:
  - This specifies the model you want to download. The format `gemma3n:e2b` indicates the specific version or variant of the `gemma3n` model, which is necessary to ensure that you are using the correct model configuration suitable for your tasks.


In [7]:
%%time
# !ollama pull gemma3n:e4b 
!ollama pull gemma3n:e2b

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIDufW3hHiuM/owH0G4XPZexespXxl4MR+38X7VuO1D1U

[GIN] 2025/06/30 - 18:42:39 | 200 |      65.696µs |       127.0.0.1 | HEAD     "/"


time=2025-06-30T18:42:39.165Z level=INFO source=routes.go:1235 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_SCHED_SPREAD:false ROCR_VISIBLE_DEVICE

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ 

time=2025-06-30T18:42:40.155Z level=INFO source=download.go:177 msg="downloading 3839a254cf2d in 16 351 MB part(s)"


pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 3839a254cf2d:   1% ▕                  ▏  45 MB/5.6 GB                  pulling manifest 
pulling 3839a254cf2d:   1% ▕                  ▏  75 MB/5.6 GB                  pulling manifest 
pulling 3839a254cf2d:   2% ▕                  ▏ 133 MB/5.6 GB                  pulling manifest 
pulling 3839a254cf2d:   3% ▕                  ▏ 190 MB/5.6 GB                  pulling manifest 
pulling 3839a254cf2d:   5% ▕                  ▏ 255 MB/5.6 GB                  pulling manifest 
pulling 3839a254cf2d:   5% ▕                  ▏ 287 MB/5.6 GB                  pulling manifest 
pulling 3839a254cf2d:   6% ▕█                 ▏ 347 MB/5.6 GB                  pulling manifest 
pulling 3839a254cf2d:   7% ▕█                 ▏ 407 MB/5.6 GB                  pulling manifest 
pulling 3839a254cf2d:   8% ▕█                 ▏ 445 MB/5.6 GB                  pulling manifest 
pulling 3839a254cf2d:   9% ▕█                 ▏ 504 MB/5.6 GB          

...

pulling manifest 
pulling 3839a254cf2d: 100% ▕██████████████████▏ 5.6 GB                         pulling manifest 
pulling 3839a254cf2d: 100% ▕██████████████████▏ 5.6 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         pulling manifest 
pulling 3839a254cf2d: 100% ▕██████████████████▏ 5.6 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         pulling manifest 
pulling 3839a254cf2d: 100% ▕██████████████████▏ 5.6 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         pulling manifest 
pulling 3839a254cf2d: 100% ▕██████████████████▏ 5.6 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         pulling manifest 
pulling 3839a254cf2d: 100% ▕██████████████████▏ 5.6 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         pulling manifest 


time=2025-06-30T18:43:09.007Z level=INFO source=download.go:177 msg="downloading 1adbfec9dcf0 in 1 8.4 KB part(s)"


pulling manifest 
pulling 3839a254cf2d: 100% ▕██████████████████▏ 5.6 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         pulling manifest 
pulling 3839a254cf2d: 100% ▕██████████████████▏ 5.6 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         pulling manifest 
pulling 3839a254cf2d: 100% ▕██████████████████▏ 5.6 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling 1adbfec9dcf0: 100% ▕██████████████████▏ 8.4 KB                         pulling manifest 
pulling 3839a254cf2d: 100% ▕██████████████████▏ 5.6 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling 1adbfec9dcf0: 100% ▕██████████████████▏ 8.4 KB                         pulling manifest 
pulling 3839a254cf2d: 100% ▕██████████████████▏ 5.6 GB                         
pulling e0a42594d802: 100% ▕██████

time=2025-06-30T18:43:10.289Z level=INFO source=download.go:177 msg="downloading a3e66f51d60b in 1 417 B part(s)"


pulling manifest 
pulling 3839a254cf2d: 100% ▕██████████████████▏ 5.6 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling 1adbfec9dcf0: 100% ▕██████████████████▏ 8.4 KB                         pulling manifest 
pulling 3839a254cf2d: 100% ▕██████████████████▏ 5.6 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling 1adbfec9dcf0: 100% ▕██████████████████▏ 8.4 KB                         
pulling a3e66f51d60b: 100% ▕██████████████████▏  417 B                         pulling manifest 
pulling 3839a254cf2d: 100% ▕██████████████████▏ 5.6 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling 1adbfec9dcf0: 100% ▕██████████████████▏ 8.4 KB                         
pulling a3e66f51d60b: 100% ▕██████████████████▏  417 B                         pulling manifest 
pulling 3839a254cf2d: 100% ▕██████████████████▏ 5.6

# 9. Gradio Integration with Gemma Model

This application uses **Gradio** to create an interactive user interface that allows users to communicate with the **Gemma 3n** model.

## Overview:

- **Gradio** is a Python library that provides an easy way to create user interfaces for machine learning models.
- It allows developers to transform their models into web applications without the need for extensive coding in web frameworks.
- Users can input questions through a text box, and the application outputs responses generated by the Gemma model in a chat format.

## Key Features:

- **Interactive Interface**: Users can ask questions and receive answers in real-time.
- **Chat History**: The application maintains a history of the conversation for better context and continuity.
- **Model Integration**: Direct integration with the Gemma model enables complex inference tasks and interactions on user input.

## How to Use The Chat:

1. Launch the application.
2. Enter a question in the provided text box.
3. Click the "Submit" button to receive a response from the Gemma model.
4. View the ongoing conversation in the chat history.

This setup makes it easier to leverage the capabilities of the Gemma model in a user-friendly manner.

In [8]:
# Chat with Gemma 3n
def ollama_chat(history, question):
    try:
        # Validate the question input
        question = question.strip()
        if not question:
            return history, "Please enter a valid question."

        # Crafting a prompt to instruct the model
        my_prompt = f"Question: {question}\n"

        # Use Ollama chat with the question only
        response = ollama.chat(model='gemma3n:e2b', messages=[
            {
                'role': 'user',
                'content': my_prompt
            },
        ])
        answer = response['message']['content']

        # Update chat history with the new interaction
        history.append(f"<div style='color: blue;'>You: {question}</div>")
        history.append(f"<div style='color: green;'>Gemma 3n: {answer}</div>")
        history_text = "<br>".join(history)
        return history_text, answer
    except Exception as e:
        return history, f"Error occurred: {str(e)}. Please try again."

# Create Gradio interface
with gr.Blocks() as demo:
    history = gr.State([])
    with gr.Column():
        gr.Markdown("# Welcome to the Cruzeta Chat Portal")
        question_input = gr.Textbox(lines=2, label="Ask your question", placeholder="E.g., What can you tell me about this image?")
        chat_output = gr.HTML(label="Chat History")
        submit_button = gr.Button("Submit")

    # Connect inputs and outputs
    submit_button.click(
        ollama_chat,
        inputs=[history, question_input],
        outputs=[chat_output, gr.Textbox(label="Response", placeholder="Model response will appear here...")]
    )

# Launch the Gradio interface
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://4884b143933ed84f6b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Example Usage of The Image Analysis:
To analyze an image:
1. Enter the URL of the image in the provided text box.
2. Type a question related to the image.
3. Click "Submit" to receive a response generated by the Gemma model.

## Conclusion:
This implementation allows for efficient image analysis using the capabilities of the Gemma model, providing users with an interactive way to inquire about image content without requiring file uploads. The combination of image URLs and natural language processing enhances the accessibility and usability of the model.

In [9]:
# Function to interact with the Gemma model
def chat_with_gemma(history, image_url, question):
    try:
        # Craft the message content with an image URL
        if image_url:
            message_content = f"You have received an image at the URL: {image_url}. Question: {question or 'Please describe the details of the image.'}"
            
            # Use the Ollama chat with the image and question
            response = ollama.chat(model='gemma3n:e2b', messages=[
                {
                    "role": "user",
                    "content": message_content
                },
            ])
            answer = response['message']['content']
        else:
            response = ollama.chat(model='gemma3n:e2b', messages=[
                {
                    "role": "user",
                    "content": f"Question: {question}"
                }
            ])
            answer = response['message']['content']

        # Update history with the new conversation turn, using HTML for styling
        history.append(f"<div style='color: blue;'>User: {question}</div>")
        history.append(f"<div style='color: green;'>Assistant: {answer}</div>")
        history_text = "<br>".join(history)
        return history_text, answer
    except Exception as e:
        return history, f"Error occurred: {str(e)}"

# Create a Gradio interface using Blocks
with gr.Blocks() as demo:
    history = gr.State([])
    with gr.Column():
        gr.Markdown("# Welcome to Cruzeta Image Analysis Portal")
        image_url_input = gr.Textbox(lines=1, label="Enter Image URL")  # Changed to accept a URL
        question_input = gr.Textbox(lines=2, label="Ask a question about the image")
        chat_output = gr.HTML(label="Chat History")
        submit_button = gr.Button("Submit")

    # Connect the inputs and outputs
    submit_button.click(
        chat_with_gemma,
        inputs=[history, image_url_input, question_input],
        outputs=[chat_output, gr.Textbox(label="Response", placeholder="Model response will appear here...")]
    )

# Launch the Gradio interface
demo.launch()

* Running on local URL:  http://127.0.0.1:7861
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://4cb1b46609a0f03397.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Example Usage of the Audio File Analysis:
To analyze an audio file:
1. Enter the URL of the audio in the provided text box.
2. Type a question related to the audio.
3. Click "Submit" to receive a response generated by the Gemma model.

## Conclusion:
This implementation allows for efficient audio analysis using the capabilities of the Gemma model, providing users with an interactive way to inquire about audio content without requiring file uploads.

In [10]:
# Gemma 3n Can Analyze Audio Files
def analyze_audio(history, audio_url, question):
    try:
        # Create the prompt for the Gemma model
        if audio_url:
            message_content = (
                f"You have received an audio file at the URL: {audio_url}. "
                f"Question: {question or 'Please describe the audio.'}"
            )
            # Send the prompt to the model using the Ollama API
            response = chat(model='gemma3n:e2b', messages=[
                {
                    "role": "user",
                    "content": message_content
                },
            ])
            answer = response['message']['content']
        else:
            # Respond only with questions if there is no audio URL
            response = chat(model='gemma3n:e2b', messages=[
                {
                    "role": "user",
                    "content": f"Question: {question}"
                }
            ])
            answer = response['message']['content']
        
        # Update the history with the new interaction
        history.append(f"<div style='color: blue;'>User: {question}</div>")
        history.append(f"<div style='color: green;'>Assistant: {answer}</div>")
        history_text = "<br>".join(history)
        
        return history_text, answer
    except Exception as e:
        return history, f"Error occurred: {str(e)}"

# Create Gradio interface
with gr.Blocks() as demo:
    history = gr.State([])
    with gr.Column():
        gr.Markdown("# Welcome to the Cruzeta Audio Analysis Portal")
        audio_url_input = gr.Textbox(lines=1, label="Enter Audio URL")  # Input for the audio URL
        question_input = gr.Textbox(lines=2, label="Ask a question about the audio")
        chat_output = gr.HTML(label="Chat History")
        submit_button = gr.Button("Submit")
        
    # Connect the inputs and outputs
    submit_button.click(
        analyze_audio,
        inputs=[history, audio_url_input, question_input],
        outputs=[chat_output, gr.Textbox(label="Response", placeholder="Model response will appear here...")]
    )

# Launch the Gradio interface
demo.launch()

* Running on local URL:  http://127.0.0.1:7862
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://ebd374378eb2ce0673.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
